In [1]:
import pennylane as qml
import numpy as np

#### Mixture of States
A mixture of states $\{\ket{i}\}_{i=1}^{N}$ prepared with probabilities $p_i$ is expressed using the density operator  
$$ \rho = \sum_{i=1}^N p_{i}\ket{i}\bra{i}$$  

Given that in the real world we cannot fully eliminate noise, this is a more realistic representaiton of the state that is fed into the quantum circuit.  
Therefore we will see how Unitary measurements and gates act on mixed states  

Like a quantum gate $U$ maps a pure state $\ket{\psi}$ as $U\ket{\psi}$ a mixed state is mapped as $$\rho \rarr{U{\rho}{U^\dagger}}$$

#### Solution to Codercise N.2.1.a
Effect of a Quantum gate $U$ on a quantum state with density operator $\rho$

In [2]:
def apply_gate_mixed(rho: np.array(np.array(complex)), U: np.array(np.array(complex))) -> np.array(np.array(complex)):
    """
    The output of this method is the density matrix of the output state after applying the quantum gate on the input state
    """
    # Multiply the input density matrix with the conjugate transpose of the matrix of the gate
    rho_u_dagger = np.matmul(rho, U.conj().T)
    
    # Multiply the gate matrix with the above computed matrix and return the result
    return np.matmul(U, rho_u_dagger)

In [3]:
"""
Test for apply_quantum_gate
"""
U = qml.matrix(qml.RX(np.pi/3,0))
rho = np.array([[3/4,1/4],[1/4,1/4]])

print("A pi/3 RX rotation applied on [[3/4,1/4],[1/4,1/4]] gives:")
print(apply_gate_mixed(rho,U).round(2))

A pi/3 RX rotation applied on [[3/4,1/4],[1/4,1/4]] gives:
[[0.62-0.j   0.25+0.22j]
 [0.25-0.22j 0.37+0.j  ]]


The same can be done using pennylane instead of numpy

In [4]:
device = qml.device('default.mixed', wires = 1) # One quantum wire, on a device for mixed states
@qml.qnode(device)
def apply_gate_circuit(rho: np.array(np.array(complex)), U: np.array(np.array(complex))) -> np.array(np.array(complex)):
    """
    The output is a state measurement as measured using pennylane
    """
     # Prepare a state with density operator rho
    qml.QubitDensityMatrix(rho, 0)
    # Apply the unitary U
    qml.QubitUnitary(U, 0)
    return qml.state() # Return the density operator after applying U


In [5]:
"""
Test for apply_gate_circuit
"""
U = qml.matrix(qml.RX(np.pi/3,0))
rho = np.array([[3/4,1/4],[1/4,1/4]])

print("A pi/3 RX rotation applied on [[3/4,1/4],[1/4,1/4]] gives:")
print(apply_gate_circuit(rho,U).round(2))


A pi/3 RX rotation applied on [[3/4,1/4],[1/4,1/4]] gives:
[[0.62+0.j   0.25+0.22j]
 [0.25-0.22j 0.38+0.j  ]]


#### Measurement of Mixed States

The above shows how to apply a quantum gate on a mixed state however we also need to _**measure**_ these mixed states.  
- Physical quantities that can be measured are encoded in observables.  
- These observables are Hermitian operators.
- Therefore these observables are diagonizable.  

Let $B$ be an observable, its corresponding Eigenvectors are $\ket{k}$ and its Eigenvalues are $\{\lambda_k\}_{k=1}^N$
* The set of values from which the result of measuring the encoded observable $B$ is given by $\lambda_k$
* The outcomes $\lambda_k$ are probabilistic, i.e. if we have a state representation given by $\rho$ then when measurement on this state is performed, the probability (denoted by $p(k)$) of obtaining $\lambda_k$ is $$ p(k) = \mathrm{tr}(\rho\Pi_k)$$ where, $$\Pi_k = \ket{k}\bra{k}$$

$\Pi_k = \ket{k}\bra{k}$ are called the _**eigenprojectors**_  
#### Solution to codercise N.2.2.a 
Computing the Eigenprojectors

In [6]:
def eigenprojectors(obs: np.array(np.array(complex))) -> np.array(np.array(complex)):
    """
    obs: is a matrix representing a hermitian operator, which represents a Quantum observable.
    ==========
    returns
    An array containing the eigenprojectors of the input observable
    """
    # Calculating the eigenvectors of the observable using numpy
    eigenvectors = np.linalg.eig(obs)
    
    # For each eigenvector, compute the outer product with its conjugate
    projectors = [np.outer(eigvec, np.conj(eigvec)) for eigvec in eigenvectors[1].T]
    
    return projectors

#### Solution to N.2.2.b
Computing the outcome probabilites when given a density operator $\rho$ and an observable $B$

In [7]:
def outcome_probs(rho: np.array(np.array(complex)), B: np.array(np.array(complex))) -> np.array(float):
    eigen_projectors = eigenprojectors(B) # Calling the function from N.2.2.a to get the eigen projections
    prob_list = [np.trace(np.matmul(p, rho)) for p in eigen_projectors] # Calculate the probabilities
    return np.array(prob_list)
    

In [8]:
"""
Test of N.2.2.a and N.2.2.b
"""
rho = np.array([[3/4,0],[0,1/4]])
B = qml.matrix(qml.PauliY(0))

print("State: [[3/4,0],[0,1/4]], Observable: {}".format(B))
print("Measurement probabilities {}".format(outcome_probs(rho,B).round(2)))

State: [[3/4,0],[0,1/4]], Observable: [[ 0.+0.j -0.-1.j]
 [ 0.+1.j  0.+0.j]]
Measurement probabilities [0.5+0.j 0.5+0.j]


The same can be done using Pennylane and not hand rolling numpy
#### Solution to N.2.2.c
* Prepare a Quantum State with the density matrix $\rho$
* Return measurement probabilities 

In [11]:
@qml.qnode(device)
def mixed_probs_circuit(rho: np.array(np.array(complex)), obs):
    qml.QubitDensityMatrix(rho, wires=0)
    return qml.probs(op=obs)

In [12]:
"""
Test for N.2.2.c
"""
rho = np.array([[3/4,0],[0,1/4]])
B = qml.PauliY(0)

print("State: [[3/4,0],[0,1/4]], Observable: {}".format(qml.matrix(B)))
print("Measurement probabilities {}".format(mixed_probs_circuit(rho,B))) 

State: [[3/4,0],[0,1/4]], Observable: [[ 0.+0.j -0.-1.j]
 [ 0.+1.j  0.+0.j]]
Measurement probabilities [0.5 0.5]
